In [21]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

In [23]:

UNIVERSE_SIZE = 100
NUM_SETS = 10
DENSITY = 0.2

# tabu search stopping criteria
MAX_ITERATIONS = 100 

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [24]:

# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

In [25]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

In [26]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)


(np.True_, np.float64(324.21015187672424))

In [27]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)


(np.False_, np.float64(175.56775028490412))

In [28]:
# generate a random initial solution as starting point
def get_initial_solution():
    return rng.random(NUM_SETS) < .5

# generate neighbors by inverting one set's values
def InvertMove(solution):
    # np.tile repeats the solution across all rows
    neighbors = np.tile(solution, (len(solution), 1))
    # invert
    for i in range(len(solution)):
        neighbors[i, i] = not neighbors[i, i]
    return neighbors
    

def tabu_search(tenure):
    current_solution = get_initial_solution()
    best_solution = current_solution.copy()
    best_cost = cost(best_solution)

    # tabu list initialized as empty
    tabu_list = []
    tabu_tenure = np.zeros(NUM_SETS)

    iteration = 0
    while iteration < MAX_ITERATIONS:
        neighbors = InvertMove(current_solution)

        # Vectorized validity check and cost calculation for all neighbors
        valid_neighbors = np.array([valid(neighbor) for neighbor in neighbors])
        neighbor_costs = np.array([cost(neighbor) for neighbor in neighbors])

        admissible_mask = (neighbor_costs < best_cost) | (iteration > tabu_tenure)
        admissible_neighbors = neighbors[valid_neighbors & admissible_mask]

        if admissible_neighbors.size == 0:
            break

        # choose best admissable neighbor
        best_neighbor = min(admissible_neighbors, key=cost)
        current_solution = best_neighbor
        current_cost = cost(current_solution)

        # update best sol if found
        if current_cost < best_cost:
            best_solution = current_solution.copy()
            best_cost = current_cost

        # update tabu list
        tabu_tenure[best_neighbor.argmax()] = iteration + tenure

        iteration += 1

    return best_solution, best_cost
         


In [29]:
# size of tabu list
tenure = 5
best_solution, best_cost = tabu_search(tenure)

print("Best solution: ", best_solution)
print("Best cost: ", best_cost)
#valid(best_solution), cost(best_cost)

Best solution:  [ True  True  True  True False False  True False  True  True]
Best cost:  231.04667682433941
